## Important

**Please use Colab Runtime Version 25.07 --> Python 3.11.13**

We will soon release czmodel = 6.0.0,which will also support newer python versions.

## Disclaimer

This content of this repository is free to use for everybody. Carl Zeiss Microscopy GmbH's ZEN software undertakes no warranty concerning the use of those scripts, image analysis settings and ZEN experiments. Use them on your own risk.

**By using any of those examples you agree to this disclaimer.**

In [ ]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [ ]:
if IN_COLAB:
    
    # Install czmodel and dependencies
    ! pip install --upgrade pip
    ! pip install "torch==2.5.1"
    ! pip install "torchvision==0.20.1"
    ! pip install "czmodel[pytorch]==5.1.0"
    ! pip install opencv-python

In [ ]:
# This can be used to switch on/off warnings
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

## Introduction
The library [czmodel](https://pypi.org/project/czmodel/) provides a base package and extras for export functionalities that require specific dependencies -:

- ```pip install czmodel``` - This would only install base dependencies, no Pytorch-specific packages.
- ```pip install czmodel[pytorch]``` - This would install base and Pytorch-specific packages.

**This is an example notebook for segmentation task with czmodel[pytorch].**

## Simple PyTorch model for segmentation (to detect cell nuclei)
This notebook shows the entire workflow of training an ANN with [PyTorch](https://pytorch.org/) and exporting the trained model to the [CZANN format](https://pypi.org/project/czmodel/) to be ready for used within ZEN & arivis software or your own python code.

* The trained model is rather simple (for demo purposes) and trained on a small test dataset.
* **Therefore, this notebook is meant to be understood as a guide for exporting trained models.**
* **The notebook does not provide instructions how train a model correctly.**

## Imports

In [ ]:
# Required imports to train a simple PyTorch model for segmentation and package it as CZANN.
# The CZANN can then be imported in ZEN and used for segmentation and image analysis workflows.

# General imports
import os
import torch
import torch.nn as nn

# Function provided by the PyPI package called czmodel (by ZEISS)
from czmodel import ModelMetadata, ModelType, LegacyModelMetadata
from czmodel.pytorch import ModelSpec, LegacyModelSpec, DefaultConverter, LegacyConverter

import logging
from typing import List
from matplotlib import pyplot as plt
import os

# This can be used to switch on/off warnings
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')


logging.getLogger("pytorch").setLevel(logging.ERROR)
print(torch.__version__)

# Use GPU if available otherwise CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

## Training Pipeline
This section describes a simple training procedure that creates a trained PyTorch model.

* Therefore, it only represents the custom training procedure
* Such procedure will vary from case to case and will contain more sophisticated ways to generate an optimized PyTorch model

### Define parameters for data loading

In [ ]:
# Folder containing the input images
IMAGES_FOLDER = 'nucleus_data/images/'

# Folder containing the ground truth segmentation masks
# Mask images have one channel with a unique value of each class (0=background and 1=nucleus)
MASKS_FOLDER = 'nucleus_data/labels/'

# Path to the data on GitHub
GITHUB_TRAINING_DATA_PATH = "https://raw.githubusercontent.com/sebi06/ZEN_Python_CZI_Smart_Microscopy_Workshop/main/workshop/notebooks/nucleus_data.zip"
GITHUB_MODEL_CONVERSION_SPEC_PATH = "https://raw.githubusercontent.com/sebi06/ZEN_Python_CZI_Smart_Microscopy_Workshop/main/workshop/notebooks/models/model_conversion_spec_pytorch.json"

# Define the number of input color channels
# This means that the inputs are grayscale with one channel only
CHANNELS = 1 

# The size of image crops to train the model with
CROP_SIZE = 512

### Download data if it's not available on disk
If this notebook is run e.g. as a colab notebook, it does not have access to the data folder on gitub via disk access. 
In that case we need to download the data from github first.

In [ ]:
import requests

# Download training data
if not (os.path.isdir(IMAGES_FOLDER) and os.path.isdir(MASKS_FOLDER)):
    compressed_data = './nucleus_data.zip'
    if not os.path.isfile(compressed_data):
        import io
        response = requests.get(GITHUB_TRAINING_DATA_PATH, stream=True)
        compressed_data = io.BytesIO(response.content)

    import zipfile
    with zipfile.ZipFile(compressed_data, 'r') as zip_accessor:
        zip_accessor.extractall('./')

# Download model conversion spec
if not os.path.isfile('./models/model_conversion_spec_pytorch.json'):

    if not os.path.isdir("./models"):
        os.mkdir("models")
    
    response = requests.get(GITHUB_MODEL_CONVERSION_SPEC_PATH, stream=True)
    with open('./models/model_conversion_spec_pytorch.json', 'wb') as handle:
        handle.write(response.content)

### Define a PyTorch dataset to pre-process the images
Since the dataset contains very large images we need to train on smaller crops in order to not exhaust the GPU memory.

In order to use the model later inside Napari, we read the test images (PNGs) as grayscale images.

In [ ]:
from torch.utils.data import DataLoader, Dataset
import torchvision
import torchvision.transforms.functional as TF
import cv2
from skimage.io import imread

# Dataset class
class SampleDataset(Dataset):
    def __init__(self, images_folder:str, masks_folder:str, transforms: torchvision.transforms=None):
        self.sample_images = sorted([os.path.join(images_folder, f) for f in os.listdir(images_folder) 
                        if os.path.isfile(os.path.join(images_folder, f))])
        self.sample_masks = sorted([os.path.join(masks_folder, f) for f in os.listdir(masks_folder) 
                       if os.path.isfile(os.path.join(masks_folder, f))])
        self.transforms = transforms

    def __getitem__(self, idx: int):
        image = self.sample_images[idx]
        mask = self.sample_masks[idx]

        # this makes sure that the PNGs are read a grayscale
        img = cv2.imread(image, cv2.IMREAD_GRAYSCALE)
        msk = cv2.imread(mask, cv2.IMREAD_GRAYSCALE)

        if self.transforms is not None:
            img = self.transforms(img)
            msk = self.transforms(msk)

        # Randomly crop the image
        i, j, h, w = torchvision.transforms.RandomCrop.get_params(img, output_size=(CROP_SIZE, CROP_SIZE))
        img = TF.crop(img, i, j, h, w)
        msk = TF.crop(msk, i, j, h, w)

        # Transform mask to one-hot encoding
        msk = msk/msk.max()
        msk = torch.nn.functional.one_hot(msk.long(), num_classes=2).squeeze().permute(2, 0, 1)

        return img.to(torch.float32), msk.to(torch.float32)

    def __len__(self):
        return len(self.sample_images)

In [ ]:
# Define transforms
from torchvision import transforms
transforms = transforms.Compose([
    transforms.ToTensor(),
])

# Create dataset and dataloader
sample_dataset = SampleDataset(images_folder=IMAGES_FOLDER,
                               masks_folder=MASKS_FOLDER,
                               transforms=transforms)

sample_dataloader = DataLoader(dataset=sample_dataset,
                                      batch_size=2,
                                      shuffle=True)

In [ ]:
# Verify data is loaded correctly
print("Data Verification:")
print(f"Total samples in dataset: {len(sample_dataset)}")
print("-" * 50)

for img, mask in sample_dataset:
    print(f"Image shape: {img.shape}, min: {img.min():.3f}, max: {img.max():.3f}")
    print(f"Mask shape: {mask.shape}, unique values: {torch.unique(mask)}")
    print(f"Mask class distribution:")
    print(f"  - Background (class 0): {(mask[0] == 1).sum().item()} pixels")
    print(f"  - Nucleus (class 1): {(mask[1] == 1).sum().item()} pixels")
    break

### Define a simple model
This part defines a PyTorch model with two convolutional layers and softmax activation at the output node. 

In [ ]:
# Define simple PyTorch model with two convolutional layers and softmax activation at the output node
class SampleModel(nn.Module):

    def __init__(self):
        super(SampleModel, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=1),
            nn.Conv2d(16, 2, 1, padding=0),
            nn.Softmax(),
        )

    def forward(self, x: torch.Tensor):
        x = self.model(x)
        return x


model = SampleModel().to(device)

### Training the model with the loaded data
This part fits the model to the loaded data. In this test example we do not care about an actual evaluation of the model using validation and test datasets.

In [ ]:
# Define the number of training epochs
NUM_EPOCHS = 30

# Define loss function and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=5e-2, eps=1e-08)

In [ ]:
def train_one_epoch(epoch_index: int):
    """Train the model for one epoch with gradient clipping."""
    model.train()
    running_loss = 0.

    for i, data in enumerate(sample_dataloader):
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # Zero gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)

        # Calculate loss
        loss = loss_fn(outputs, labels)
        
        # Backward pass
        loss.backward()

        # Gradient clipping (prevents exploding gradients)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        # Update weights
        optimizer.step()
        
        running_loss += loss.item()

    return running_loss / len(sample_dataloader)

In [ ]:
# define plotting function
def plot_history(loss: List):
    plt.plot(loss, "*-")
    plt.title('training loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.show()

In [ ]:
# run the simple training and show the loss with better monitoring
training_loss = []

print("Starting training...")
print(f"Device: {device}")
print(f"Number of training samples: {len(sample_dataset)}")
print(f"Batch size: {sample_dataloader.batch_size}")
print(f"Learning rate: {optimizer.param_groups[0]['lr']}")
print("-" * 50)

for epoch in range(NUM_EPOCHS):
    loss = train_one_epoch(epoch)
    training_loss.append(loss)
    
    # Print progress every 10 epochs
    if (epoch + 1) % 5 == 0 or epoch == 0:
        print(f'Epoch {epoch+1:3d}/{NUM_EPOCHS}, Loss: {loss:.6f}')

print("-" * 50)
print(f"Final loss: {training_loss[-1]:.6f}")
print(f"Initial loss: {training_loss[0]:.6f}")
print(f"Loss reduction: {training_loss[0] - training_loss[-1]:.6f}")

# Plot the training history
plot_history(training_loss)

In [ ]:
def predict_img(net,
                img,
                device):
    net.eval()
    img = img.unsqueeze(0)
    img = img.to(device=device, dtype=torch.float32)

    with torch.no_grad():
        output = net(img).cpu()

        mask = output.argmax(dim=1)
        
        pred = mask[0].long().squeeze().numpy()

    return pred

In [ ]:
import matplotlib.pyplot as plt

model.to(device=device)

# get an image and a mask from the dataset
for img, mask in sample_dataset:
    
    # run the prediction
    pred = predict_img(net=model,
                       img=img,
                       device=device)
    
    # show some results
    img_scaled = img.numpy()[0] / img.numpy().max()
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3)
    fig.set_figwidth(16)

    # show images and set titles 
    ax1.imshow(img_scaled)
    ax2.imshow(pred, cmap="gray", vmin=0, vmax=1)
    ax3.imshow(mask[0,...])

    ax1.set_title('Image')
    ax2.set_title('Prediction')
    ax3.set_title('Label Mask')

In [ ]:
# Move the model to cpu and save it to disk
if not os.path.isdir("./models"):
    os.mkdir("models")
PATH = './models/saved_torch_segmentation_model.pt'
model.cpu()
torch.save(model, PATH)

## Create a CZANN/CZMODEL from the trained PyTorch model
In this section we export the trained model to the CZANN format using the czmodel library and some additional meta data all possible parameter choices are described in the [ANN model specification](https://pypi.org/project/czmodel/).

### Define Meta Data
We first define the meta data needed to run the model within the Intellesis infrastructure. The `czmodel` package offers a named tuple `ModelMetadata` that allows to either parse as JSON file as described in the [specification document](https://pypi.org/project/czmodel/) or to directly specify the parameters as shown below.

### Create a Model Specification Object
The export functions provided by the `czmodel` package expect a `ModelSpec` tuple that features the PyTorch model to be exported, the corresponding model meda data and optionally a license file for the model.

Therefore, we wrap our model and the `model_metadata` instance into a `ModelSpec` object.

In [ ]:
# Define dimensions - ZEN or Arivis Pro require fully defined spatial dimensions in the meta data of the CZANN model.
# The TilingClient uses the input shape in the meta data to infer the tile size to pass an image to the inferencer.
# Important: The tile size has to be chosen s.t. inference is possible with the minimum hardware requirements
# Optional: Define target spatial dimensions of the model for inference.

input_size = 512

# Define the model metadata
model_metadata = ModelMetadata(
    input_shape=[input_size, input_size, 1],
    output_shape=[input_size, input_size, 2],
    model_type=ModelType.SINGLE_CLASS_SEMANTIC_SEGMENTATION,
    classes=["Background", "Nucleus"],
    model_name="Simple_Nuclei_SegmentationModel",
    min_overlap=[64, 64],
    scaling=(1.0, 1.0),
)
model_spec = ModelSpec(
    model=model,
    model_metadata=model_metadata,
    license_file=None
)

### Perform model export into *.czann / *.czmodel file format

The converters from the `czmodel` library offers two functions to perform the actual export. 

* `convert_from_json_spec` allows to provide a JSON file containing all the information of a ModelSpec object and converts a model in PyTorch model format on disk to a `.czann` / `.czmodel` file that can be loaded with ZEN.
* `convert_from_model_spec` expects a `ModelSpec` object, an output path and name and the input shape of the exported model. From this information it creates a `.czann` / `.czmodel` file containing the specified model.

Currently, `czmodel` offers two converters:
* DefaultConverter: Converts a model to a *.czann file.
* LegacyConverter: Converts a model to a *.czmodel file (legacy format).

We will only use the `DefaultConverter` here.

In [ ]:
DefaultConverter().convert_from_model_spec(
    model_spec=model_spec, 
    output_path='./models', 
    output_name='simple_pytorch_nuclei_segmodel_pytorch',
    input_shape=(1, input_size, input_size),
)

# In the example above there will be a "./simple_nuclei_segmodel_pytorch.czann" file saved on disk.

## Remarks
The generated .czann and .czmodel files can be directly loaded into ZEN Intellesis to perform segmentation tasks with the trained model.
If there is already a trained model in PyTorch model format present on disk, it can also be converted by providing the path to the saved model directory instead of a PyTorch `Module` object. The `czmodel` library will implicitly load the model from the provided path.

The `czmodel` library also provides a `convert_from_json_spec` function that accepts a JSON file with the above mentioned meta data behind the key `ModelMetadata` which will implicitly be deserialized into a `ModelMetadata` object, the model path and optionally a license file:

```json
{
    "ModelMetadata": {
        "Type": "SingleClassSemanticSegmentation",
        "Classes": ["Background", "Nucleus"],
        "InputShape": [512, 512, 1],
        "OutputShape": [512, 512, 2],
        "ModelName": "Nuclei Segmentation Model From JSON",
        "MinOverlap": [64, 64],
        "Scaling": [1.0, 1.0]
    },
    "ModelPath": "./models/saved_torch_segmentation_model.pt",
    "LicenseFile": null
}
```

This information can be copied to a file e.g. in the current working directory `./models/model_conversion_spec_pytorch.json` that also contains the trained model in SavedModel format e.g. generated by the following line:

In [ ]:
# This is an additional way to create a CZANN from a saved PyTorch model on disk + JSON file.
# The currently recommended way to to create the CZANN directly by using czmodel.convert_from_model_spec
# the path to the saved Pytorch model is defined in the JSON shown above

DefaultConverter().convert_from_json_spec(
    model_spec_path='./models/model_conversion_spec_pytorch.json',
    output_path='./models',
    output_name = 'simple_nuclei_segmodel_from_json_pytorch',
    input_shape=(1, input_size, input_size),
)